# Vulture

## Introduction to Text Operations with Vulture

In [1]:
import os
import pickle
import pathlib
import pandas as pd

from TELF.pre_processing import Vulture
from TELF.pre_processing.Vulture.modules import AcronymDetector
from TELF.pre_processing.Vulture.modules import SimpleCleaner
from TELF.pre_processing.Vulture.tokens_analysis.levenstein import replace_similar_keys_levenshtein


## 0. Load Dataset

### Input

In [2]:
DATA_DIR = os.path.join('..', '..', 'data')
DATA_DIR = pathlib.Path(DATA_DIR).resolve()
DATA_FILE = 'acronyms_documents.p'
documents = pickle.load(open(os.path.join(DATA_DIR, DATA_FILE), 'rb'))
len(documents)

11

In [3]:
documents

{'ID_1': "In our discussion about Global Warming (GW), we should also consider Sustainable Energy Sources (SES). While discussing SES, it's vital to mention Carbon Footprint Reduction (CFR) and its impacts on Environmental Sustainability (ES). Let's not forget the role of Innovative Technology Solutions (ITS).",
 'ID_2': "The project's success hinges on Effective Team Collaboration (ETC), robust Project Management Frameworks (PMF), and the use of Advanced Analytics Tools (AAT). Furthermore, Continuous Improvement Processes (CIP) and Strategic Planning Initiatives (SPI) are key.",
 'ID_3': "Digital Transformation (DT) involves more than just technology. It's about Customer Engagement Strategies (CES) and optimizing the User Experience (UX). Moreover, Data Privacy Regulations (DPR) and Cybersecurity Measures (CM) are foundational.",
 'ID_4': 'Financial institutions are focusing on Risk Management Practices (RMP) and Investment Diversification Strategies (IDS). The importance of Regulator

### Output

In [4]:
RESULTS_DIR = 'results'
RESULTS_DIR = pathlib.Path(RESULTS_DIR).resolve()
RESULTS_FILE = 'operated_documents'
try:
    os.mkdir(RESULTS_DIR)
except FileExistsError:
    pass

### Setup Vulture

Create a single-node multi-process Vulture object

In [5]:
vulture = Vulture(n_jobs  = 1, 
                  verbose = 10,  # Disable == 0, Verbose >= 1
                 )
steps = [SimpleCleaner( 
            order = [
                'standardize_hyphens',
                'isolate_frozen',
                'remove_copyright_statement',
                'remove_stop_phrases',
                'make_lower_case',
                'remove_formulas',
                'normalize',
                'remove_next_line',
                'remove_email',
                'remove_()',
                'remove_[]',
                'remove_special_characters',
                'remove_nonASCII_boundary',
                'remove_nonASCII',
                'remove_tags',
                'remove_stop_words',
                'remove_standalone_numbers',
                'remove_extra_whitespace',
                'min_characters',
        ])]
%time vulture.clean(documents, \
                    steps=steps,\
                    save_path=os.path.join(RESULTS_DIR, "clean_documents"))         

clean_documents = pickle.load(open(os.path.join(RESULTS_DIR, "clean_documents"), 'rb'))

[Vulture]: Cleaning 11 documents
  0%|          | 0/1 [00:00<?, ?it/s][Vulture]: Running SimpleCleaner module



  0%|          | 0/11 [00:00<?, ?it/s]/Users/maksim/Desktop/Code/telf_internal/TELF/pre_processing/Vulture/modules/simple_clean.py:290: RuntimeWarning: [SimpleCleaner]: Requested stop phrase removal but no stop phrases were provided!
  warnings.warn('[SimpleCleaner]: Requested stop phrase removal but no '
/Users/maksim/Desktop/Code/telf_internal/TELF/pre_processing/Vulture/modules/simple_clean.py:249: RuntimeWarning: [SimpleCleaner]: Requested stop word removal but no stop words were provided!
  warnings.warn('[SimpleCleaner]: Requested stop word removal but no '
100%|██████████| 1/1 [00:00<00:00, 76.79it/s]

CPU times: user 10 ms, sys: 18.7 ms, total: 28.8 ms
Wall time: 43.7 ms


In [6]:
vulture.operate(clean_documents, steps=[AcronymDetector(replace_raw=True)], save_path=RESULTS_DIR, file_name=RESULTS_FILE)                   

[Vulture]: Cleaning 11 documents
100%|██████████| 1/1 [00:00<00:00, 43.51it/s]


Each entry is a tuple where index 0 is the name of the operation and index 1 is the results of the operation in dictionary format.

In [7]:
saved_file = ! ls $RESULTS_DIR
saved_file

['clean_documents',
 'clean_documents.p',
 'operated_documents_AcronymDetector.p']

### Look at Cleaned Documents

In [8]:
operated_documents = pickle.load(open(os.path.join(RESULTS_DIR, 'operated_documents_AcronymDetector.p'), 'rb'))
operated_documents

{'ID_1': {'Acronyms': {'global warming': 'gw',
   'environmental sustainability': 'es',
   'sustainable energy sources': 'ses',
   'carbon footprint reduction': 'cfr',
   'innovative technology solutions': 'its'},
  'replaced_text': 'in our discussion about  global_warming we should also consider  sustainable_energy_sources while discussing sustainable_energy_sources innovative_technology_solutions vital to mention  carbon_footprint_reduction and innovative_technology_solutions impacts on  environmental_sustainability lets not forget the role of  innovative_technology_solutions'},
 'ID_2': {'Acronyms': {'effective team collaboration': 'etc',
   'project management frameworks': 'pmf',
   'advanced analytics tools': 'aat',
   'continuous improvement processes': 'cip',
   'strategic planning initiatives': 'spi'},
  'replaced_text': 'the projects success hinges on  effective_team_collaboration robust  project_management_frameworks and the use of  advanced_analytics_tools furthermore  conti

In [9]:
def to_df(documents, operated_documents):
    data = {
        'id': [],
        'text': [],
        'acronyms': [],
        'acronym_replaced_text': [],

    }

    for _id, text in documents.items():
        data['id'].append(_id)
        data['text'].append(text)

        data['acronyms'].append(operated_documents.get(_id).get('Acronyms'))
        data['acronym_replaced_text'].append(operated_documents.get(_id).get('replaced_text'))

    return pd.DataFrame.from_dict(data)

In [10]:
df = to_df(documents, operated_documents)
df

,id,text,acronyms,acronym_replaced_text
0,ID_1,"In our discussion about Global Warming (GW), w...","{'global warming': 'gw', 'environmental sustai...",in our discussion about global_warming we sho...
1,ID_2,The project's success hinges on Effective Team...,"{'effective team collaboration': 'etc', 'proje...",the projects success hinges on effective_team...
2,ID_3,Digital Transformation (DT) involves more than...,"{'digital transformation': 'dt', 'cybersecurit...",digital_transformation involves more than jus...
3,ID_4,Financial institutions are focusing on Risk Ma...,"{'regulatory compliance': 'rc', 'risk manageme...",financial institutions are focusing on risk_m...
4,ID_5,"In healthcare, Patient Care Coordination (PCC)...","{'patient care coordination': 'pcc', 'electron...",in healthcare patient_care_coordination and ...
5,ID_6,Educational Technology (ET) is reshaping learn...,"{'educational technology': 'et', 'interactive ...",educational_technology is reshaping learning ...
6,ID_7,The entertainment industry values Creative Con...,"{'marketing strategies': 'ms', 'creative conte...",the entertainment industry values creative_co...
7,ID_8,Sustainable Agriculture Practices (SAP) and Pr...,"{'sustainable agriculture practices': 'sap', '...",sustainable_agriculture_practices and precis...
8,ID_9,Urban Planning (UP) incorporates Green Infrast...,"{'urban planning': 'up', 'green infrastructure...",urban_planning incorporates green_infrastruc...
9,ID_10,"To Be Determined (TBD), National Aeronautics a...","{'as soon': 'as', 'to be determined': 'tbd', '...",to_be_determined national aeronautics and spa...


In [11]:
acronyms_list = df.acronyms.to_list()
almost_correct_acronyms = [{'effective team collaboratioyy': 'etc'},
 {'audience engcgement metricsy': 'aem'},
 {'strategic nlanning initiativesy': 'spi'},
 {'global warmingy': 'gw'},
 {'environmental sustainabilityy': 'es'},
 {'sustainable energy sourcesy': 'ses'},
 {'carbon footprint reductiony': 'cfr'},
 {'innovative technology solutionsy': 'its', 'innnovative technology solutions': 'itsss'},
 {'project management frameworksy': 'pmf'},
 {'advanced analytics toolsy': 'aat'},
 {'continuous improvement processesy': 'cip'}]

acronyms_list += almost_correct_acronyms
acronyms_list

[{'global warming': 'gw',
  'environmental sustainability': 'es',
  'sustainable energy sources': 'ses',
  'carbon footprint reduction': 'cfr',
  'innovative technology solutions': 'its'},
 {'effective team collaboration': 'etc',
  'project management frameworks': 'pmf',
  'advanced analytics tools': 'aat',
  'continuous improvement processes': 'cip',
  'strategic planning initiatives': 'spi'},
 {'digital transformation': 'dt',
  'cybersecurity measures': 'cm',
  'customer engagement strategies': 'ces',
  'data privacy regulations': 'dpr'},
 {'regulatory compliance': 'rc',
  'risk management practices': 'rmp',
  'investment diversification strategies': 'ids',
  'financial technology innovations': 'fti',
  'consumer financial protection': 'cfp'},
 {'patient care coordination': 'pcc',
  'electronic health records': 'ehr',
  'medical research innovations': 'mri',
  'health information exchanges': 'hie',
  'public health initiatives': 'phi'},
 {'educational technology': 'et',
  'interactiv

In [12]:
changes_path = os.path.join(RESULTS_DIR,'levenshtein_changes.csv')
modified_acronyms_list = replace_similar_keys_levenshtein(acronyms_list, changes_made_save_path=changes_path)

chunk_size = 14, num_cpus = 12, len all_pairs = 163


Processing Chunks: 100%|██████████| 12/12 [00:00<00:00, 5673.73it/s]


In [13]:
len(acronyms_list),len(modified_acronyms_list[0])

(22, 22)

In [14]:
df_changes = pd.read_csv(changes_path)
df_changes

,Previous Key,New Key,Similarity Score
0,carbon footprint reduction,carbon footprint reductiony,0.962963
1,environmental sustainability,environmental sustainabilityy,0.965517
2,innovative technology solutions,innovative technology solutionsy,0.968750
3,sustainable energy sources,sustainable energy sourcesy,0.962963
4,advanced analytics tools,advanced analytics toolsy,0.960000
5,continuous improvement processes,continuous improvement processesy,0.969697
6,project management frameworks,project management frameworksy,0.966667
7,innnovative technology solutions,innovative technology solutions,0.968750
8,innovative technology solutions,innovative technology solutionsy,0.968750
